# **Car Detection Project**

In [2]:
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from scipy.ndimage import label
import collections

%matplotlib inline

# **Extracting Features**

In [3]:
# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features  
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualize=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualize=vis, feature_vector=feature_vec)
        return features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256),  orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0, vis=False):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = cv2.imread(file)
        # apply color conversion to YCrCb
        feature_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
   
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        
        # Call get_hog_features() with vis=False, feature_vec=True
        
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        
        # Append the new feature vector to the features list
        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    # Return list of feature vectors
    return features

# **Define parameters**


In [4]:
# Define HOG parameters
orient = 12
pix_per_cell = 8
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

# Spatial size and histogram parameters
spatial_size=(16, 16)
hist_bins=16

# **Extract features and make prediction**

In [12]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    draw_img = np.copy(img)
    draw_img_all_windows = np.copy(img)
    img = img.astype(np.float32)/255

    
    bbox_list = []
    
    img_tosearch = img[ystart:ystop,:,:]
    ####################################
    # img_tosearch_temp = cv2.putText(np.copy(img_tosearch), 'img size = {}'.format(img_tosearch.shape), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    # cv2.imshow('img_tosearch_temp', img_tosearch_temp)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    ####################################
    ctrans_tosearch =  cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2YCrCb)
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int32(imshape[1]/scale), np.int32(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]
    
    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell)-1
    nyblocks = (ch1.shape[0] // pix_per_cell)-1 
    nfeat_per_block = orient*cell_per_block**2
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell)-1 
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))

            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))    
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            #test_prediction = svc.predict(test_features)
            test_prediction = svc.decision_function(test_features)
 
            xbox_left = np.int32(xleft*scale)
            ytop_draw = np.int32(ytop*scale)
            win_draw = np.int32(window*scale)
            cv2.rectangle(draw_img_all_windows,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6)
            # cv2.imshow('draw_img_all_windows', draw_img_all_windows)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            
            #if test_prediction == 1:
            if test_prediction > 0.4:
                xbox_left = np.int32(xleft*scale)
                ytop_draw = np.int32(ytop*scale)
                win_draw = np.int32(window*scale)
                bbox_list.append(((xbox_left, ytop_draw + ystart), (xbox_left + win_draw, ytop_draw + win_draw + ystart)))
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                
                # str_on_img_1 = 'xbox_left: {} - ytop_draw+ystart: {} - xbox_left+win_draw: {}'.format(xbox_left, ytop_draw+ystart, xbox_left+win_draw)
                # str_on_img_2 = 'ytop_draw+win_draw+ystart: {}'.format(ytop_draw+win_draw+ystart)
                # img = cv2.putText(draw_img, str_on_img_1, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                # img = cv2.putText(img, str_on_img_2, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                # cv2.imshow('img', img)
                # cv2.waitKey(0)
                # cv2.destroyAllWindows()
                
    return bbox_list, draw_img, draw_img_all_windows

def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
        
        # Visualizing the heatmap
        # cv2.imshow('heatmap', heatmap)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    bbox_ = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,255,0), 6)
        bbox_.append(bbox)
    # Return the image
    return img, bbox_

# **Import classifier model and feature extraction settings**

In [6]:
import pickle
# Load info
dist_pickle = pickle.load( open("classifier_info.p", "rb" ) )
svc_l = dist_pickle["svc"]
X_scaler_l = dist_pickle["scaler"]
orient_l = dist_pickle["orient"]
pix_per_cell_l = dist_pickle["pix_per_cell"]
cell_per_block_l = dist_pickle["cell_per_block"]
spatial_size_l = dist_pickle["spatial_size"]
hist_bins_l = dist_pickle["hist_bins"]


# **Test on a video**

In [7]:
def process_image(img):
    heatmaps = collections.deque(maxlen=29)
    # global heatmaps

    heat = np.zeros_like(img[:,:,0]).astype(np.float32)
    
    # ystarts = [400,350,350]
    # ystops = [656,570,570]
    ystarts = [400]
    ystops = [656]
    # ystarts = [200,175,175]
    # ystops = [328,285,285]
    
    # Look for cars at different scales
    # scales = [1., 1.5, 2.0]
    # scales = [1.0, 2.0]
    scales = [1.5]
    for scale, ystart, ystop  in zip(scales, ystarts, ystops):
        box_list, out_img, out_img_windows  = find_cars(img, ystart, ystop, scale, svc_l, X_scaler_l, orient_l, pix_per_cell_l, cell_per_block_l, spatial_size_l, hist_bins_l)
        heat = add_heat(heat,box_list)
    # Append heatmap and compute the sum of the last n ones
    heatmaps.append(heat)
    sum_heatmap = np.array(heatmaps).sum(axis=0)
    # Apply the threshold to remove false positives
    # heat = apply_threshold(sum_heatmap, min(len(heatmaps) * 1, 28))
    print(min(len(heatmaps) * 1, 28))
    heat = apply_threshold(sum_heatmap, 0)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img, bbox = draw_labeled_bboxes(np.copy(img), labels)
    
    # visualizing heatmaps in numbers
    # print('threshold = {}'.format(min(len(heatmaps) * 1, 28)))
    # cv2.imshow('sum_heatmap: ', sum_heatmap)
    # cv2.imshow('thresholded heatmap: ', heat)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return draw_img, bbox



# **Running the classifier**

In [13]:
src_path = 'videos/project_video_large.mp4'
result_path = 'videos/output/output_project_video_large.avi'
# result_path = 'detection_results_scale_only_1.5_y_scanned_256_modulo_5_semi_final_V.1.avi'

# openCV's configurations to save the video on disk.
vid = cv2.VideoCapture(src_path)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(result_path, codec, fps, (width, height))

accepted_frame = 0
frame_count = 0
bboxes = []
while True:
    return_value, frame = vid.read()
    if return_value:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if accepted_frame == 0:
            frame, bboxes = process_image(frame)
            # frame = cv2.putText(frame, 'Obtained bbox', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        elif bboxes:
            # frame = cv2.putText(frame, 'Interpolated bbox', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            for bbox in bboxes:
                frame = cv2.rectangle(frame, bbox[0], bbox[1], (0,255,0), 6)
        accepted_frame = (accepted_frame + 1) % 5
        frame_count += 1
        print("Frame number: ", frame_count)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # if frame_count > 100:
        #     cv2.imshow('frame no.{}'.format(frame_count), frame)
        #     cv2.waitKey(0)
        #     cv2.destroyAllWindows()
        # elif frame_count == 1:
        #     break
        out.write(frame)
        
    else:
        print('Video ended or an error has occurred.')
        break


1
Frame number:  1
Frame number:  2
Frame number:  3
Frame number:  4
Frame number:  5
1
Frame number:  6
Frame number:  7
Frame number:  8
Frame number:  9
Frame number:  10
1
Frame number:  11
Frame number:  12
Frame number:  13
Frame number:  14
Frame number:  15
1
Frame number:  16
Frame number:  17
Frame number:  18
Frame number:  19
Frame number:  20
1
Frame number:  21
Frame number:  22
Frame number:  23
Frame number:  24
Frame number:  25
1
Frame number:  26
Frame number:  27
Frame number:  28
Frame number:  29
Frame number:  30
1
Frame number:  31
Frame number:  32
Frame number:  33
Frame number:  34
Frame number:  35
1
Frame number:  36
Frame number:  37
Frame number:  38
Frame number:  39
Frame number:  40
1
Frame number:  41
Frame number:  42
Frame number:  43
Frame number:  44
Frame number:  45
1
Frame number:  46
Frame number:  47
Frame number:  48
Frame number:  49
Frame number:  50
1
Frame number:  51
Frame number:  52
Frame number:  53
Frame number:  54
Frame number:  